In [1]:
import os
os.chdir("../../")

In [2]:
import scanpy as sc
import numpy as np
import pandas as pd

In [2]:
!Rscript Toy.R

Error in library(splatter) : there is no package called ‘splatter’
Execution halted


In [3]:
counts_df = pd.read_csv("./data/toy/toy.csv")
counts_df = counts_df.transpose()
counts_df.head()

,Gene1,Gene2,Gene3,Gene4,Gene5,Gene6,Gene7,Gene8,Gene9,Gene10,...,Gene991,Gene992,Gene993,Gene994,Gene995,Gene996,Gene997,Gene998,Gene999,Gene1000
Cell1,69,376,4,34,50,2,156,4,13,7,...,7,3,4,0,54,0,0,40,20,0
Cell2,31,396,3,40,41,0,134,4,3,0,...,8,1,1,1,33,0,1,25,11,0
Cell3,50,425,3,39,28,0,133,5,27,6,...,3,12,0,3,64,0,0,30,50,0
Cell4,68,426,0,51,23,0,254,4,9,4,...,6,1,3,0,39,2,0,16,48,0
Cell5,29,292,1,8,17,1,104,2,2,0,...,2,0,2,0,14,0,0,8,24,1


In [4]:
batches_df = pd.read_csv("./data/toy/toy_batch.csv")
batches_df['x'].value_counts()

Batch8    4000
Batch5    4000
Batch6    4000
Batch7    4000
Batch3    4000
Batch2    4000
Batch1    4000
Batch9    4000
Batch4    4000
Name: x, dtype: int64

In [5]:
celltypes_df = pd.read_csv("./data/toy/toy_celltype.csv")
celltypes_df['x'].value_counts()

Group5    7281
Group2    7265
Group1    7226
Group4    7161
Group3    7067
Name: x, dtype: int64

In [6]:
counts_df.shape, batches_df.shape, celltypes_df.shape

((36000, 1000), (36000, 1), (36000, 1))

In [7]:
counts_adata = sc.AnnData(X=counts_df.values)
counts_adata.obs['batch'] = batches_df.values
counts_adata.obs['celltype'] = celltypes_df.values
counts_adata.var_names = counts_df.columns
counts_adata

AnnData object with n_obs × n_vars = 36000 × 1000 
    obs: 'batch', 'celltype'

In [8]:
counts_adata.write_h5ad("./data/toy/toy_count.h5ad")

... storing 'batch' as categorical
... storing 'celltype' as categorical


In [8]:
normalized_adata = counts_adata.copy()

In [9]:
sc.pp.normalize_per_cell(normalized_adata)
sc.pp.log1p(normalized_adata)

In [10]:
normalized_adata.X.min(), normalized_adata.X.max()

(0.0, 7.288168)

In [11]:
normalized_adata.write_h5ad("./data/toy/toy_normalized.h5ad")